In [16]:
import os 
import pandas as pd
import argparse
import sys
import h5py
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from functions_aa import *

raw_df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQwC6jRtVUk-2dkk2W3BDJZTOdsS427LN8Ixo-rQF4Afs6ice0rof7qh_EbnAy5lYEGqX-TCSvjpPyr/pub?gid=1713335339&single=true&output=csv',
                     index_col=['codigo','Tiempo']).drop(['Fecha','Exposicion'], axis=1)

raw_df.sort_index(inplace=True)
unique_IDs = raw_df.index.get_level_values(0).unique().values
n_IDs      = unique_IDs.__len__()
train_size = .9
test_size  = .1
train_set_ids, val_set_ids = train_test_split(unique_IDs, train_size=.8)
_,            test_set_ids = train_test_split(val_set_ids, train_size=train_size)

print(train_set_ids, val_set_ids, test_set_ids, train_set_ids.__len__(), val_set_ids.__len__(), test_set_ids.__len__())


/DeepenData/.miniconda/envs/torch_aa/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Cuda is ready
['T05' 'T06' 'T15' 'T16' 'T04' 'T17' 'T08' 'T12' 'T13' 'T14' 'T18' 'T19'
 'T10' 'T09' 'T07'] ['T01' 'T11' 'T03' 'T02'] ['T03'] 15 4 1


In [11]:
timepoints_per_patient = pd.Series(raw_df.index.get_level_values(0).values).value_counts()
smallest_window        = timepoints_per_patient.min()
df                     = raw_df.loc[(slice(None), slice(1, smallest_window)), :]
train_set = df.loc[train_set_ids]
val_set   = df.loc[val_set_ids]
test_set  = df.loc[test_set_ids]
assert all(train_set.index.get_level_values(0).unique().to_list() == train_set_ids)
assert all(val_set.index.get_level_values(0).unique().to_list() == val_set_ids)
assert all(test_set.index.get_level_values(0).unique().to_list() == test_set_ids)

In [14]:
def process_each_set(set_df,smallest_window):
    sample_ids    = set_df.index.get_level_values(0).to_numpy().reshape(-1, smallest_window)[:, 0]
    feature_names = set_df.columns.tolist()
    X             = set_df.to_numpy()
    X             = X.reshape(len(sample_ids), smallest_window, len(feature_names))
    return X, feature_names

train_set_X, _ = process_each_set(train_set, smallest_window)
val_set_X,  _              = process_each_set(val_set, smallest_window)
test_set_X,  _             = process_each_set(test_set, smallest_window)


In [18]:
artificial_missing_rate = .1
train_set_dict = add_artificial_mask(train_set_X, artificial_missing_rate, 'train')
val_set_dict   = add_artificial_mask(val_set_X, artificial_missing_rate, 'val')
test_set_dict  = add_artificial_mask(test_set_X, artificial_missing_rate, 'test')
processed_data = {

    'train': train_set_dict,
    'val': val_set_dict,
    'test': test_set_dict
}

In [19]:
dataset_saving_dir = f"{os.getcwd()}/data"

saving_into_h5(dataset_saving_dir, processed_data, classification_dataset=False)
print(f'All done. Saved to {dataset_saving_dir}.')

All done. Saved to /DeepenData/Repos/Tyrosinemia/data.


In [20]:
original_data_dfs = {'train_set_df':train_set,
                    'val_set_df':val_set,
                    'test_set_df':test_set,}


pickle_path = os.getcwd()+'/data/datasets.pickle'

import pickle


with open(pickle_path, 'wb') as handle:
    pickle.dump(original_data_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)